# Week 5: Task 13
#### Name: Kai Ferragallo-Hawkins
#### Date: 20.2.2023

##Setup



In [108]:
import pandas as pd
import numpy as np
import samplics as smp
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import SurveySamplingFunctions as ssf

### Functions
def cluster_descr(df, cluster_column, sorting_column):
    """Given a Pandas dataframe, a cluster column, and a sorting column, find descriptors for all clusters. Implement random and limited cluster selection and working regardless of what value you consider a cluster in the future?"""
    cluster_descriptors = pd.DataFrame()
    clusters = sorted(df[cluster_column].unique())
    for cluster_num in range(1, len(clusters)+1):
        cluster_stats = df.loc[df[cluster_column] == cluster_num, sorting_column].describe()
        cluster_descriptors[f'Cluster {clusters[cluster_num-1]}'] = cluster_stats
    return cluster_descriptors

def system_var (main, sample, sample_key, Is_Sqrt = False):
    """"Calculates the system variance by manually calculating the information for design variance based on population sizes, given two pandas dataframes of the original information and the sample. Population variance calculated through Pandas var."""
    sqr = 0.5 if Is_Sqrt else 1
    s_var = ((len(main)**2)*(1-len(sample)/len(main))*(1/len(sample))*sample[sample_key].var(ddof=1))**sqr
    return s_var

### Province 91
## Importing Province91 Data
province91 = pd.read_csv("files/assignment1/province91.txt", delim_whitespace=True)
# Mean notably above the median for UE91, so creating new stratum that separates UE91 by the mean.
province91['UE91 Above Mean'] = (province91['UE91'] > province91['UE91'].mean()).astype(int)
display(province91)

### Province 17
## Importing Province17 Data
province17 = pd.read_csv("files/assignment1/province17.txt", delimiter = '\t', encoding='latin-1')

,Stratum,Cluster,Id,Municipality,POP91,LAB91,UE91,HOU85,URB85,UE91 Above Mean
0,1,1,1,Jyväskylä,67200,33786,4123,26881,1,1
1,1,2,2,Jämsä,12907,6016,666,4663,1,1
2,1,2,3,Jämsänkoski,8118,3818,528,3019,1,1
3,1,2,4,Keuruu,12707,5919,760,4896,1,1
4,1,3,5,Saarijärvi,10774,4930,721,3730,1,1
5,1,5,6,Suolahti,6159,3022,457,2389,1,0
6,1,3,7,Äänekoski,11595,5823,767,4264,1,1
7,2,5,8,Hankasalmi,6080,2594,391,2179,0,0
8,2,6,9,Joutsa,4594,2069,194,1823,0,0
9,2,7,10,Jyväskmlk,29349,13727,1623,9230,0,1


## Task 13: Cluster Sampling
This tasks are as follows:
-  Calculate descriptive statistics for 1) whole province91 data for UE91 and 2) by clusters (variable Cluster) for UE91.
The cluster descriptors show a wide variety of possible values, without a clear or decisive ordering. This comes from the fact that the clusters are presumably fully randomized. Cluster 1 also shows how outlier data (i.e. a value of 4123 for unemployed) can completely change the descriptors.

Normal UE91 descriptors have a mean smaller than 4 of the clusters and larger then the other 4 clusters, a decent preliminary sign for the effectiveness of the cluster samplings distribution.

In [109]:
### Province 91
## Descriptive Statistics (UE91)
UE91_descriptors = province91['UE91'].describe()
print("UE91 Descriptors")
display(UE91_descriptors)

## Descriptive Statistics (Clusters)
cluster_descriptors_org = cluster_descr(province91, "Cluster", "UE91")
print("Original Cluster Descriptors")
display(cluster_descriptors_org)

UE91 Descriptors


count      32.000000
mean      471.812500
std       743.402918
min        54.000000
25%       128.750000
50%       229.000000
75%       538.000000
max      4123.000000
Name: UE91, dtype: float64

Original Cluster Descriptors


,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8
count,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,1206.00000,535.250000,427.250000,171.500000,480.750000,109.000000,555.750000,289.000000
std,1945.39439,250.957334,367.135193,116.162243,283.739758,65.518445,714.017448,209.066178
min,166.00000,187.000000,79.000000,94.000000,201.000000,54.000000,119.000000,128.000000
25%,220.75000,442.750000,126.250000,97.000000,343.500000,59.250000,194.000000,128.750000
50%,267.50000,597.000000,431.500000,125.500000,424.000000,94.000000,240.500000,230.000000
75%,1252.75000,689.500000,732.500000,200.000000,561.250000,143.750000,602.250000,390.250000
max,4123.00000,760.000000,767.000000,341.000000,874.000000,194.000000,1623.000000,568.000000


- Use cluster sampling and draw your own samples.
I choose to draw 4 random clusters by creating a list with an even distribution of values 1-4 and then randomly shuffling their placement. The end result appears to be three smaller clusters and one larger cluster.

In [110]:
## Draw Own Cluster Samples
# Randomizing cluster values from 1 to 4
np.random.seed(123)
randomized_task13 = [1, 2, 3, 4]*(len(province91)//4)
np.random.shuffle(randomized_task13)

# Creating cluster list in code
province91['Cluster_Task13'] = randomized_task13

# Getting Cluster descriptors
cluster_descriptors_task13 = cluster_descr(province91, "Cluster_Task13", "UE91")
print("Task 13 Cluster Descriptors")
display(cluster_descriptors_task13)

Task 13 Cluster Descriptors


,Cluster 1,Cluster 2,Cluster 3,Cluster 4
count,8.000000,8.000000,8.000000,8.000000
mean,320.625000,332.875000,290.250000,943.500000
std,274.370416,270.117137,244.243292,1378.191465
min,61.000000,54.000000,98.000000,94.000000
25%,138.500000,160.000000,125.000000,177.750000
50%,202.500000,275.000000,177.000000,359.500000
75%,436.000000,425.250000,364.000000,905.250000
max,767.000000,874.000000,760.000000,4123.000000


- Estimate UE91 total and deff.
The resulting total UE91 follows the understanding from the above code, with three of the clusters having a smaller total value then the actual UE91 and one (4) having a larger total value. The Deff values continuous increase feels odd - variation may correlate, but I don't believe they should not be exact. More work to ensure that the deff values are correct is needed. However, if taken as they are, only the cluster 4 is less efficient then the simple random sampling base.

In [111]:
## UE91 Total Estimation
# Ordering by clusters
ue91_totals_nw = province91.groupby('Cluster_Task13')['UE91'].sum()

# Creating individual cluster weights (all the same)
cluster_weights = len(province91) / province91.groupby('Cluster_Task13').size()

# Creating UE91 total list
ue91_totals_es = ue91_totals_nw * cluster_weights
df_ue91_adj_totals = pd.DataFrame({'UE91 Estimated Totals': ue91_totals_es})
df_ue91_adj_totals.reset_index(inplace=True)

## Calculate deff value
# SRS Sample + Variation
prov91_sample = province91.sample(n=8, replace=False, random_state=123456)
s_var91_srs = system_var(province91, prov91_sample, "UE91", Is_Sqrt = True)

# CLU Variation
s_var91_clu = []
s91_deff = []
for num in range(4):
    s_var91_clu.append(system_var(province91, province91[province91['Cluster_Task13'] == num+1], "UE91", Is_Sqrt = True))
    s91_deff.append(s_var91_clu[num]/s_var91_srs)

# Table Display
df_ue91_adj_totals["Deff Values"] = s91_deff
display(df_ue91_adj_totals)



,Cluster_Task13,UE91 Estimated Totals,Deff Values
0,1,10260.0,0.542980
1,2,10652.0,0.534562
2,3,9288.0,0.483358
3,4,30192.0,2.727443
